# Inverse geometry

Inverse geometry is the problem of computing a configuration $q$ of a robot model that satisfies a set of 'tasks', which we formalize as error functions $e(q)$ that are zero when the task is achieved.

- $q \in \mathcal{C}$: configuration of the robot.
- $e_i(q) \in \mathbb{R}$: error function for task $i$. In this tutorial our error function is real valued.

Typical tasks for limbed robots include putting a foot/end-effector on a surface, moving the center of mass (CoM) to a target location, etc. For example, an error function for a foot position task could be:

$$
e(q) = \|p_\mathrm{foot}^* - p_\mathrm{foot}(q)\|
$$

In this tutorial, we will define a task function $e(q)$ for the end-effector pose of a manipulator and solve the inverse geometry problem

$$
\underset{q}{\mathrm{minimize}}\ e(q)
$$

using an off-the-shelf nonlinear optimizer that evaluates gradients by finite differences (slower), then a more efficient formulation using CasADi (faster).

## Before we start...

(The following line was used to generate the notebook, you can skip it:)

In [27]:
import magic_donotload

Let's import everything we will need in this tutorial:

In [28]:
# %load tp1/generated/invgeom_imports
import time
import unittest

import example_robot_data as robex
import meshcat_shapes
import numpy as np
import pinocchio as pin
from numpy.linalg import norm
from scipy.optimize import fmin_bfgs



## Set up

Let us start by loading the UR5 robot model from `example-robot-data` and define its neutral configuration as follows:

In [29]:
# %load tp1/generated/invgeom_robot
robot = robex.load("ur5")
robot.q0 = np.array([0, -np.pi / 2, 0, 0, 0, 0])
model = robot.model
data = robot.data


We will also use the MeshCat visualizer to watch the optimization unfold. Let's set up a visualizer for our robot model:

In [30]:
# %load tp1/generated/invgeom_visualizer
viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model
)
robot.setVisualizer(viz, init=False)
robot.setVisualizer(viz, init=False)
viz.initViewer(open=False)
viz.loadViewerModel()
viz.display(robot.q0)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


## Part 1: BFGS with finite difference

In this first part, we will define an inverse geometry problem for the UR5 and use the BFGS optimizer from [`scipy.optimize.fmin_bfgs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_bfgs.html#scipy-optimize-fmin-bfgs) to solve it.

### Task target

Our task will be to move the end effector *frame* of the UR5 to a desired pose (pose := position + orientation). For this purpose, we define:

- The name of the corresponding frame, which is ``"tool0"`` for the UR5.
- The target pose of this frame, which we define by an affine transform ${}_{world} M_{target} \in SE(3)$ (also known as a "placement").

The robot will start from an arbitrary initial configuration (hint: to make the solver's life easier, avoid the singular $[0\ 0\ \ldots\ 0]$ configuration) $q_0$.

In [31]:
# %load tp1/generated/invgeom_task_params
tool_id = model.getFrameId("tool0")

transform_target_to_world = pin.SE3(
    pin.utils.rotate("x", np.pi / 4),
    np.array([-0.5, 0.1, 0.2]),
)


### Task error

For a given robot configuration $q$, the end effector will be located at ${}_{world} M_{tool}(q)$, which is a function of $q$ we can compute with Pinocchio's forward kinematics function. The goal of our task will be to make ${}_{world} M_{tool}(q)$ coincide with ${}_{world} M_{target}$, which we can define mathematically as: minimize the (Lie-group) difference ${}_{world} M_{target} \ominus {}_{world} M_{tool}(q)$, or equivalently:

$$
\underset{q}{\mathrm{minimize}} \ e(q) := \| {}_{world} M_{target} \ominus {}_{world} M_{tool}(q) \|
$$

Remember the definition of the $\ominus$ (left) Lie-group difference? No? Let's write it in Python anyway!

In [32]:
# %load tp1/generated/invgeom_error_function

def error(q: np.ndarray) -> float:
    """Function that the final configuration should minimize."""
    pin.framesForwardKinematics(model, data, q)
    transform_tool_to_world = data.oMf[tool_id]
    return norm(
        pin.log(
            transform_tool_to_world.inverse() * transform_target_to_world
        ).vector
    )




### Sidetrack: visualization



In [33]:
# %load tp1/generated/invgeom_visualizer_callback
viewer = viz.viewer
meshcat_shapes.frame(viewer["target"], opacity=1.0)
meshcat_shapes.frame(viewer["current"], opacity=0.5)


def callback(q: np.ndarray):
    pin.framesForwardKinematics(model, data, q)
    transform_frame_to_world = data.oMf[tool_id]
    viewer["target"].set_transform(transform_target_to_world.np)
    viewer["current"].set_transform(transform_frame_to_world.np)
    viz.display(q)
    time.sleep(1e-1)




We are now ready to optimize for real!

### BFGS optimization

In [35]:
# %load tp1/generated/invgeom_fmin_bfgs
qguess = np.array([0.12, -2.2, -1.45, 1.82, -0.95, 0.17])
qopt = fmin_bfgs(error, qguess, callback=callback)

print(
    "The robot finally reached effector placement at\n",
    robot.placement(qopt, 6),
)


         Current function value: 0.000000
         Iterations: 70
         Function evaluations: 964
         Gradient evaluations: 136
The robot finally reached effector placement at
   R =
           1 -1.35995e-08  -4.3225e-09
-1.26728e-08    -0.707107    -0.707107
 6.55981e-09     0.707107    -0.707107
  p =     -0.5 0.158195 0.141805



If everything went well, our robot converged to a configuration $q$ that achieves the desired end effector placement, *i.e.* $e(q) \approx 0 \Leftrightarrow {}_{world} M_{target} \approx {{}_{world} M_{tool}}(q)$.

## Part 2: IPOPT with automatic differentiation

We wil now define our problem in [CasADi](https://web.casadi.org), a framework that allows us to formulate problems as computation graphs from which it can compute derivatives (notably first-order derivatives, *a.k.a.* gradients, and second-order derivatives, *a.k.a.* Hessians) automatically. This feature is called *automatic differentiation*.

In [37]:
# %load tp1/generated/invgeom_casadi_imports
import casadi
from pinocchio import casadi as cpin



Pinocchio, in the version 3 we are using in today's tutorial, provides a `pinocchio.casadi` interface to represent configurations (such as $q \in \mathcal{C}$ or $M \in SE(3)$) in CasADi computation graphs, and differentiate them appropriately. (This is not as trivial as representing everything as matrices and vectors in $\mathbb{R}^n$ and taking their usual derivatives; here Pinocchio allows us to take derivatives over manifolds properly, without worrying about what happens below the surface.) To be able to do that, we first need to redefine our model as a CasADi one:

In [ ]:
# %load tp1/generated/invgeom_casadi_model
cmodel = cpin.Model(model)
cdata = cmodel.createData()
